### Restart and Run All Cells

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2023
quarter = 3
today = date.today()
today_str = today.strftime("%Y-%m-%d")
today_str

'2023-08-10'

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [3]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2023 AND quarter <= 3) 
OR (year = 2023-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC



In [8]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.sample(4)

,name,year,quarter,q_amt,Counter
129,OISHI,8091,10,1234225,4
70,FPT,8091,10,1742063,4
191,TFFIF,8091,10,2035986,4
121,MC,8091,10,901548,4


In [9]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2023 AND quarter <= 3-1) 
OR (year = 2023-1 AND quarter >= 3) 
ORDER BY year DESC, quarter DESC


In [10]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.sample(5).style.format(format_dict)

,name,year,quarter,q_amt,Counter
83,HMPRO,8090,10,"6,417,062",4
72,GC,8090,10,"163,925",4
92,INTUCH,8090,10,"10,917,236",4
10,AOT,8090,10,"-1,336,443",4
104,KSL,8090,10,"1,530,380",4


In [11]:
dfp.name.unique().shape

(231,)

In [12]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(231, 15)

In [13]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(399, 9)

In [14]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.sample(5).style.format(format_dict)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [15]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'
"""
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM qt_profits 
WHERE year = 2023 AND quarter = 'Q3'



0

In [16]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.sample(5)

,name,id
398,ZEN,689
52,BH,61
23,AQUA,29
292,SMPC,455
342,TIPH,529


In [17]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.sample(5).style.format(format_dict)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [18]:
rcds = df_ins.values.tolist()
len(rcds)

4

In [19]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, \
inc_amt, inc_pct, ticker_id) \
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [20]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [22]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].sample(4).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
2,OISHI,2023,Q3,"1,234,225","1,187,041","47,184",3.97%
1,MC,2023,Q3,"901,548","847,819","53,729",6.34%
0,FPT,2023,Q3,"1,742,063","2,033,577","-291,514",-14.34%
3,TFFIF,2023,Q3,"2,035,986","1,996,055","39,931",2.00%


In [23]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].sample().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
2,OISHI,2023,Q3,"1,234,225","1,187,041","47,184",3.97%


In [25]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent


In [26]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent


In [27]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id


In [28]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id


In [29]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id


In [30]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Group of 4 Quarters
